# Understanding Delta Lake Transactions

To provide atomic, durable transactions in the Lakehouse, Delta Lake utilizes transaction logs stored alongside data files. This notebook dives into the transaction logs to demonstrate what information is recorded and explore how the current version of the table is materialized.

## Learning Objectives
By the end of this lessons, student will be able to:
- Describe how Delta Lake tracks and manages transactions
- Define the atomicity and durability guarantees of Delta Lake
- Map transactional metadata back to table operations

## Setup
Run the following script to setup necessary variables and clear out past runs of this notebook.

In [0]:
%run ../Includes/Classroom-Setup-1.3

## Atomic, Durable Transactions in the Lakehouse

Briefly, Delta Lake transactions can be described as the following:
- All write operations commit data changes as Parquet files
- Transactions commit when JSON log files are written
- Logs are stored in a nested directory
- Both data and transaction logs inherit the durability guarantees of the file system

The cloud-based object storage used by Databricks provides the following advantages and guarantees:
- Infinitely scalable
- Affordable
- Availability:  > 99.9%
- Durability: > 99.999999999%

Note that the Databricks platform has SLAs separate from cloud vendor infrastructure, and the guarantees of each cloud vendor differ slightly.

## Create a Table and Insert Records

The cell below contains 4 Delta Lake operations.

In [0]:
%sql

CREATE TABLE bronze 
(id INT, name STRING, value DOUBLE); 

INSERT INTO bronze VALUES (1, "Yve", 1.0);
INSERT INTO bronze VALUES (2, "Omar", 2.5);
INSERT INTO bronze VALUES (3, "Elia", 3.3);

As expected, the table has three rows.

In [0]:
%sql
SELECT * FROM bronze

Reviewing the history allows us to see how operations and versions are linked.

In [0]:
%sql
DESCRIBE HISTORY bronze

The transaction log directory is nested under the table directory and contains a log file for each transaction (alongside checksum files and other files used to manage consistency with cloud object storage).

In [0]:
files = dbutils.fs.ls(f"{DA.paths.user_db}/bronze/_delta_log")
display(files)

The table was created without any data present, so the initial version only contains metadata about the table schema and the user, environment, and time of the transaction.

Note that the transaction log is a simple JSON file, which can be reviewed directly from the file or read with Spark.

In [0]:
display(spark.read.json(f"{DA.paths.user_db}/bronze/_delta_log/00000000000000000000.json"))

The cell below leverages a Databricks setting to automatically return the Delta transaction log for the most recent commit version.

In [0]:
def display_delta_log(table, version=None):
    if not version:
        version = spark.conf.get("spark.databricks.delta.lastCommitVersionInSession")
    version_str = str(int(version)).zfill(20)
    file = f"{DA.paths.user_db}/{table}/_delta_log/{version_str}.json"
    print("Showing: "+file)
    display(spark.read.json(file))

Here we'll see that a simple insert transaction results in new files being tracked in the **`add`** column alongside **`commitInfo`**.

In [0]:
display_delta_log("bronze")

## Appending Data
Note that when appending data, multiple data files may be written as part of a single transaction.

In [0]:
%sql
INSERT INTO bronze
VALUES (4, "Ted", 4.7),
       (5, "Tiffany", 5.5),
       (6, "Vini", 6.3)

The **`add`** column below contains paths and stats for each of the files added to the table.

In [0]:
display_delta_log("bronze")

Most append operations will insert many records into a single file.

In [0]:
%sql
INSERT INTO bronze SELECT * FROM new_records

As expected, multiple records appear in a single data file under the **`add`** column.

In [0]:
display_delta_log("bronze")

## Deleting Data
Deleting a single record from a file with multiple records will actually result in a file being added.

In [0]:
%sql 
DELETE FROM bronze WHERE name = "Viktor"

The file added contains the other records that were in the same data file as the record deleted in the transaction. 

In this specific case, the **`remove`** column indicates that the previous file with 3 records is no longer valid; the **`add`** column points to a new file containing only 2 records.

This is the expected behavior, as Delta Lake does not modify data files in place.

In [0]:
display_delta_log("bronze")

## Updating Data
Anytime a record in an existing file is modified, a new file will be added, and the old file will be indicated in the **`remove`** column, as in the update below.

In [0]:
%sql
UPDATE bronze SET name = "Vincent" WHERE id = 6

Here, both the file removed and the file rewritten contain only the modified record. In production environments, you are unlikely to see a data file containing a single record.

In [0]:
display_delta_log("bronze")

## Combining Transactions with Merge
The Delta Lake **`MERGE`** syntax allows updates, deletes, and inserts to occur in a single transaction.

In [0]:
%sql
MERGE INTO bronze b
USING updates u
ON b.id=u.id
WHEN MATCHED AND u.type = "update"
  THEN UPDATE SET *
WHEN MATCHED AND u.type = "delete"
  THEN DELETE
WHEN NOT MATCHED AND u.type = "insert"
  THEN INSERT *

Looking at the table history, note how **`MERGE`** operations differ from **`DELETE`**, **`UPDATE`**, or **`INSERT`** transactions.

In [0]:
%sql
DESCRIBE HISTORY bronze

The **`operationMetrics`** and **`operationParameters`** within the **`commitInfo`** describe the source data, the parameters for the query, and all the results of the transaction.

Use the query below to find:
- How many rows were in the **`updates`** table
- How many records were inserted
- How many records were deleted
- How many records were updated
- The total number of files removed
- The total number of files added

In [0]:
display_delta_log("bronze")

## Table Utility Functions
Delta Lake table utility functions modify the metadata of a table without changing underlying data.

In [0]:
%sql
ALTER TABLE bronze
ALTER COLUMN name
COMMENT "User first name"

Files will not be marked as added or removed, as only the metadata has been updated.

In [0]:
display_delta_log("bronze")

## File Compaction
Running **`OPTIMIZE`** on a table will compact small files toward the target file size.

In [0]:
%sql
OPTIMIZE bronze

During file compaction, many files should be marked as removed, while a smaller number of files will be marked as added.

No data or files are deleted during this operation; as always, adding files to the **`remove`** column means they will not be used when querying the current version of the table, but does not permanently delete them from the underlying storage.

In [0]:
display_delta_log("bronze")

## Transaction Log Checkpoints
Databricks will automatically create Parquet checkpoint files at fixed intervals to accelerate the resolution of the current table state.

Version 10 of the table should have both a **`.json`** and a **`.checkpoint.parquet`** file associated with it.

In [0]:
files = dbutils.fs.ls(f"{DA.paths.user_db}/bronze/_delta_log")
display(files)

Rather than only showing the operations of the most recent transaction, this checkpoint file condenses all of the **`add`** and **`remove`** instructions and valid **`metaData`** into a single file.

This means that rather than loading many JSON files and comparing files listed in the **`add`** and **`remove`** columns to find those data files that currently represent the valid table version, a single file can be loaded that fully describes the table state.

Transactions after a checkpoint leverage this starting point, resolved new info from JSON files with the instructions from this Parquet snapshot.

In [0]:
display(spark.read.parquet(f"{DA.paths.user_db}/bronze/_delta_log/00000000000000000010.checkpoint.parquet"))

Note that all of the data files in both the **`add`** and **`remove`** columns are still present in the table directory.

In [0]:
files = dbutils.fs.ls(f"{DA.paths.user_db}/bronze")
display(files)

## Cleaning Up Stale Data Files
Executing **`VACUUM`** performs garbage cleanup on this directory. By default, a retention threshold of 7 days will be enforced; here it is overridden to demonstrate permanent removal of data. Manually setting **`spark.databricks.delta.vacuum.logging.enabled`** to **`True`** ensures that this operation is also recorded in the transaction log. 

**NOTE**: Vacuuming a production table with a short retention can lead to data corruption and/or failure of long-running queries.

In [0]:
spark.conf.set("spark.databricks.delta.vacuum.logging.enabled", True)
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", False)
spark.sql("VACUUM bronze RETAIN 0 HOURS")
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", True)

As expected, a single file remains.

In [0]:
files = dbutils.fs.ls(f"{DA.paths.user_db}/bronze")
display(files)

Note that both the start and end of the **`VACUUM`** operation are recorded in the history.

In [0]:
%sql
DESCRIBE HISTORY bronze

The **`VACUUM START`** version will record the number of files to be deleted, but does not contain a list of file names.

Once deleted, previous versions of the table relying on these files are no longer accessible.

In [0]:
display_delta_log("bronze", 11)

Additional reading is available in this <a href="https://databricks.com/blog/2019/08/21/diving-into-delta-lake-unpacking-the-transaction-log.html" target="_blank">blog post</a>.

Run the following cell to delete the tables and files associated with this lesson.

In [0]:
DA.cleanup()